# 🌿 Global Spice Intelligence AI Assistant
## Introduction & Overview
Welcome to the world of Global Spice Intelligence — an AI-powered wellness assistant designed to bridge the gap between ancient wisdom and modern technology.

This project introduces a conversational AI that offers personalized guidance on the use of spices and natural remedies for health and beauty. By blending traditional healing systems with cutting-edge AI tools, it empowers users to explore natural wellness in a safe, intelligent, and accessible way.

### ✨ What Can This Assistant Do?
* **Understand Your Needs:**
It analyzes your input to detect health concerns, beauty goals, and even mentions of medications.

* **Recommend Natural Remedies:**
Based on systems like Ayurveda, Traditional Chinese Medicine, and other global traditions, it suggests herbs and spices that align with your needs.

* **Explain the Science:**
It shares evidence-based insights about the properties, benefits, and traditional uses of each recommended spice.

* **Pull From the Web (When Needed):**
If the internal database doesn’t have an answer, the assistant can tap into web-based sources to offer additional, up-to-date knowledge.

* **Put Safety First:**
While the assistant is smart, it always reminds you to consult with a medical professional for serious conditions or before making health decisions.

In [ ]:
# Cell 1: Install Dependencies
# Install the google-genai SDK.
!pip install -qU 'google-genai==1.7.0'

In [ ]:
# Cell 2: Import Libraries 
# Main Bot
import pandas as pd
import sqlite3
import spacy
import os

# Grounding
from google import genai
from google.genai import types

# Image and Document Understanding
from PIL import Image
import io

# Import Kaggle Seccret
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
# Cell 3: List available Gemini models
for model in client.models.list():
    print(model.name)

### 🌞 Agents and Their Functions
To bring this intelligent wellness assistant to life, we’ve designed a system of specialized AI agents — each with a unique role. These agents collaborate to understand user input, retrieve relevant knowledge, and generate meaningful responses.

Let’s take a closer look at how each one contributes to the experience.

In [ ]:
# Cell 4: Base Agent Class
class BaseAgent:
    """Base class for all agents"""
    def __init__(self, name):
        self.name = name

    def process(self, input_data):
        raise NotImplementedError("Subclasses must implement this method")

### 🧠 InputAgent
The `InputAgent` is the detective of the AI assistant, carefully analyzing your input to understand your needs. Here's how it works:

* **Listens to Your Concerns:**
  It pays close attention to the words you use, whether you're describing symptoms, medications you're taking, or beauty goals you have.
  
* **Deciphers the Meaning:**
  It uses natural language processing (NLP) techniques to identify key entities and concepts in your message, like specific health issues or ingredients you're interested in.

* **Creates a Structured Summary:**
  It transforms your raw input into a structured format, organizing the information into categories like symptoms, medications, and beauty needs. This makes it easier for other agents to understand and respond to your query.

👉 Think of this as the attentive listener who asks clarifying questions and summarizes your concerns to ensure everyone is on the same page.

In [ ]:
# Cell 5: Input Agent
class InputAgent:
    def analyze_text(self, text):
        """Analyze user input for symptoms, medications, and beauty needs."""
        analysis = {
            'symptoms': [],
            'drugs': [],
            'beauty_need': False,
            'raw_text': text,
        }

        text_lower = text.lower()

        # Symptom detection
        symptoms_mapping = {
            'headache': ['headache', 'head pain', 'migraine'],
            'stomach pain': ['stomach pain', 'stomachache', 'belly ache'],
            'dry skin': ['dry skin', 'skin dryness', 'flaky skin']
        }

        for symptom, keywords in symptoms_mapping.items():
            if any(keyword in text_lower for keyword in keywords):
                analysis['symptoms'].append(symptom)

        # Medication detection
        medications = ['aspirin', 'ibuprofen', 'paracetamol']
        for med in medications:
            if med in text_lower:
                analysis['drugs'].append(med)

        # Beauty need detection
        beauty_keywords = ['beauty', 'skin care', 'skin', 'face', 'hair', 'cosmetic']
        if any(keyword in text_lower for keyword in beauty_keywords):
            analysis['beauty_need'] = True


        return analysis

### 💊 RecommendationAgent
The `RecommendationAgent` is the heart of generating personalized spice and remedy suggestions. Here's how it works:

* **Analyzes User Needs:**
  It takes the structured data from the InputAgent—symptoms, medications, and beauty goals—as its starting point.
  
* **Consults the Knowledge Base:**
  It taps into a database of traditional healing practices and spice properties to find relevant information.
  
* **Tailors Recommendations:**
  Based on the analysis, it generates a list of spices and remedies that are likely to be helpful for the user's specific situation.

* **Prioritizes Safety:** It considers potential interactions with medications and includes warnings when necessary, putting user well-being first.
  
👉 Think of this as the wise herbalist who carefully selects the right ingredients to create a custom blend just for you.

In [ ]:
# Cell 6: Recommendation Agent
class RecommendationAgent:
    def generate_recommendations(self, analysis):
        """Generate spice and remedy recommendations based on analysis."""
        recommendations = {
            'spices': [],
            'remedies': [],
            'warnings': []
        }

        # Spice and remedy database
        recommendation_db = {
            'headache': {
                'spices': ['turmeric', 'ginger'],
                'remedies': ['ginger tea', 'peppermint oil massage']
            },
            'stomach pain': {
                'spices': ['fennel', 'peppermint'],
                'remedies': ['chamomile tea', 'warm lemon water']
            },
            'dry skin': {
                'spices': ['cinnamon', 'turmeric'],
                'remedies': ['honey face mask', 'coconut oil application']
            },
            'beauty': {
                'spices': ['turmeric', 'saffron'],
                'remedies': ['yogurt and honey mask', 'aloe vera gel']
            }
        }

        # Generate recommendations based on symptoms
        for symptom in analysis['symptoms']:
            if symptom in recommendation_db:
                recommendations['spices'].extend(recommendation_db[symptom]['spices'])
                recommendations['remedies'].extend(recommendation_db[symptom]['remedies'])

        # Add beauty recommendations if needed
        if analysis['beauty_need']:
            recommendations['spices'].extend(recommendation_db['beauty']['spices'])
            recommendations['remedies'].extend(recommendation_db['beauty']['remedies'])

        # Add warnings if medications are mentioned
        if analysis['drugs']:
            meds_list = ", ".join(analysis['drugs'])
            recommendations['warnings'].append(
                f"Consult a doctor before using spices with {meds_list}"
            )

        # Remove duplicates
        recommendations['spices'] = list(set(recommendations['spices']))
        recommendations['remedies'] = list(set(recommendations['remedies']))

        return recommendations

### 🤖 ResponseAgent
The `ResponseAgent` is the voice of the AI assistant, crafting clear and helpful replies for the user. Here's how it works:

* **Transforms Data into Dialogue:**
  It takes the raw recommendations from the RecommendationAgent and translates them into natural, easy-to-understand language.
  
* **Prioritizes Clarity and Safety:**
  It formats the recommendations clearly, adds warnings and disclaimers where necessary, ensuring user safety and understanding.

* **Connects to the Web (When Needed):**
  If the system's internal knowledge is insufficient, the ResponseAgent uses Gemini Grounding to search trusted online sources, providing broader insights.

* **Offers a Human Touch:**
  It aims to deliver responses that are informative, supportive, and feel more like a conversation with a wellness coach than a rigid AI interaction.

👉 Think of this as the friendly guide who explains the recommendations, answers your questions, and ensures you feel comfortable and informed.

In [ ]:
# Cell 7: Response Agent and the Grounding function
class ResponseAgent:
    def generate_response(self, recommendations, analysis):
        """Generate a user-friendly response from recommendations."""
        response = []

        if recommendations['spices']:
            response.append("🌿 Recommended spices: " + ", ".join(recommendations['spices']))

        if recommendations['remedies']:
            response.append("💡 Natural remedies: " + ", ".join(recommendations['remedies']))

        if recommendations['warnings']:
            response.append("⚠️ " + "\n⚠️ ".join(recommendations['warnings']))

        if not any(recommendations.values()): # Check if all recommendation lists are empty
            config_with_search = types.GenerateContentConfig(
                tools=[types.Tool(google_search=types.GoogleSearch())],
            )
            # ... (Same fallback_prompt as before) ...

            fallback_prompt = (
            f"You are a Global Traditional Medicine Suggestion Bot. You will Based on the user's query: '{analysis['raw_text']} to offer offer suggestions rooted in traditional healing systems "
            f"from around the world, including Ayurveda, Traditional Chinese Medicine, African Herbalism, Native American remedies, and others. "
            f"You can only discuss topics related to these healing traditions, their remedies, and the wisdom they offer in nutrition, beauty, and wellness. "
            f"Do not offer modern medical advice or make medical diagnoses. You are not a substitute for a healthcare provider, and you must not recommend anything outside the scope of traditional practices.\n\n"
            f"Based on this input, identify relevant symptoms, any mentioned herbs or therapies, and the cultural context if available. "
            f"Generate a culturally appropriate list of traditional remedies or practices that may be helpful. "
            f"These can include food-based remedies, breathing techniques, herbal concoctions, or wellness rituals, as long as they are rooted in traditional practice. "
            f"Remind the user that these are not medical treatments and always advise consulting certified traditional healers or health professionals as needed. "
            f"Do not try to continue the conversation after you finished giving a whole response. "
            )

            try:
                response_web = client.models.generate_content(
                    model='gemini-2.0-flash',
                    contents=fallback_prompt,
                    config=config_with_search,
                )
                rc = response_web.candidates[0]
                response.append(f"I couldn't find specific spice recommendations in my database, but here's something from the web:\n\n{rc.content.parts[0].text}")
            except Exception as e:
                response.append("I'm having trouble retrieving web-based information right now. Please try again later.")


        if not response:
            response.append("I recommend consulting a healthcare professional for personalized advice.")

        return "\n".join(response)

### 🧪 The Chatbox
Because Kaggle does not support raw input saving within the notebook, we’ve commented out the main chat interface code by default.

👉 **To try it out:**
Please scroll down to the code cell under **# Uncomment the whould chunk below to activate the chatbox** and uncomment it to activate the main functionality of our AI assistant.

This will allow you to enter your own questions and receive real-time personalized suggestions based on your health needs!

In [ ]:
# Cell 8: Spice AI Assistant
# Run with text input mode
# Uncomment the whould chunk below to activate the chatbox
"""
class SpiceAIAssistant:
    def __init__(self):
        self.input_agent = InputAgent()
        self.recommendation_agent = RecommendationAgent()
        self.response_agent = ResponseAgent()


if __name__ == "__main__":
    assistant = SpiceAIAssistant()
    print("Welcome to Global Spice Intelligence AI Assistant!")
    print("How can I help you with health or beauty needs today?")
    print("Examples:\n- 'I have a headache'\n- 'Natural remedies for dry skin'\n- 'What helps with stomach pain?'")

    while True:
        try:
            text = input("\n> ").strip()
            if text.lower() in ['exit', 'quit', 'stop', 'bye']:
                print("Goodbye! Stay healthy!")
                break
            if not text:
                print("Please describe your health or beauty concern")
                continue

            analysis = assistant.input_agent.analyze_text(text)

            print("\nAnalysis:")
            if analysis['symptoms']:
                print(f"- Detected symptoms: {', '.join(analysis['symptoms'])}")
            if analysis['drugs']:
                print(f"- Mentioned medications: {', '.join(analysis['drugs'])}")
            if analysis['beauty_need']:
                print("- Beauty-related request detected")

            recommendations = assistant.recommendation_agent.generate_recommendations(analysis)
            response = assistant.response_agent.generate_response(recommendations, analysis)

            print("\nAI Recommendation:")
            print(response)

        except KeyboardInterrupt:
            print("\nGoodbye! Hope I was helpful!")
            break
        except Exception as e:
            print(f"\nError: {e}\nLet's try again...")
"""

### 📄 Document Understanding
Powered by Google Gemini 2.0 Flash, the AI excels at interpreting complex documents. Here's how it works:

* **Unveils Traditional Knowledge:**
  It dives into documents rich in specialized jargon—like ancient texts or scientific studies—and extracts key insights.
  
* **Organizes Information:**
  It categorizes the extracted knowledge by tradition, remedy, use case, and preparation method, making it easy to navigate.

* **Makes Knowledge Actionable:**
  It transforms complex information into clear, concise summaries that everyday users can understand and apply.

👉 Think of this as a scholar who translates ancient wisdom into a language you can use today.

In [ ]:
# Cell 9: Document Understanding and Analyzing
def analyze_text_document(document_path):
    """Process traditional medicine documents with correct model name"""
    try:
        with open(document_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        try:
            with open(document_path, 'r', encoding='latin-1') as file:
                text = file.read()
        except Exception as e:
            return f"Error reading file: {str(e)}"


    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=[text],
        config=types.GenerateContentConfig(
            system_instruction=(

                "Extract key traditional medicine knowledge from this document, "
                "organizing by: tradition, remedy, use case, preparation. "

            ),
            temperature=0.3,
            max_output_tokens=2000
        )
    )

    return response.text
text_result=analyze_text_document("/kaggle/input/tcm-medicine/dong.txt")

print(text_result)

### 🎨 Image Understanding
Powered by Google Gemini 2.0 Flash, the AI delivers high-performance visual recognition. Here's how it works:

* **Sees Beyond the Surface:**
  It analyzes images of herbs, spices, and medicinal plants, even recognizing subtle details and variations.
  
* **Identifies Key Elements:**
  It pinpoints the plant or material in the image, linking it to its cultural origins and traditional uses.

* **Provides Contextual Information:**
  It shares insights into preparation methods, potential benefits, and safety considerations.

👉 Think of this as a botanist who can identify a plant by its appearance and tell you its secrets.

In [ ]:
# Cell 10: Image Understanding and Analyzing
def analyze_medicine_image(image_path):
    """Process traditional medicine images with correct model name"""
    try:
        try:
            img = Image.open(image_path)
        except Exception as e:
            return f"Error opening image: {str(e)}"

        # Initialize the vision model

        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=[img],
            config=types.GenerateContentConfig(
                system_instruction=(
                    "Analyze this image of traditional medicine and provide:\n"
                    "1. Identification of any visible plants/materials\n"
                    "2. Likely cultural origins/traditions\n"
                    "3. Common medicinal uses\n"
                    "4. Preparation methods if discernible\n"
                    "5. Any safety considerations\n"
                    "Format your response clearly with these headings"
                ),
                temperature=0.3,
                max_output_tokens=2000
            )
        )

        return response.text

    except Exception as e:
        return f"Error analyzing image: {str(e)}"

# Example usage:
img_result = analyze_medicine_image("/kaggle/input/tcm-medicine/images.jpeg")
print(img_result)

### 💡 Future Improvements
To enhance the AI assistant's capabilities and user experience, we envision the following improvements:

* **User-Friendly Front-End:**
  A dedicated interface will make the assistant more accessible and intuitive for all users.

* **Multilingual Support:**
  Expanding language options will broaden the assistant's reach and cultural relevance.
  
* **Advanced GenAI Enhancements:**
  Incorporating cutting-edge AI technologies will enhance response accuracy and personalization.